In [2]:
import urllib.request
import time
import sys
import getopt
import pandas as pd
import numpy as np
import pickle


In [28]:
%run ../utils.ipynb

In [3]:
embSize = 300
d1_emb_size=20
d2_emb_size=20
trainFile='../data/GAD_merged_samples_mesh.csv'

# Replace with path of word embdding file   
wefile = "../Dataset/embeddings/crawl-300d-2M.vec"
random_seed=1331

In [8]:
import pandas as pd
from tabulate import tabulate
ftrain = '../data/GAD_merged_samples_mesh.csv'
with open(ftrain, 'r', encoding='latin1') as file:
    first_line = file.readline()
    print(first_line)
import pandas as pd

ftrain = '../data/GAD_merged_samples_mesh.csv'

# Specify the delimiter for tabs
df = pd.read_csv(ftrain)


# Display the first 10 rows of the DataFrame as a table
print(tabulate(df.head(4), headers='keys', tablefmt='grid'))


GAD_ID,associationType,geneSymbol,GAD_GENE_NAME,geneId,gene_mention,GENE_ENTITY_OFFSET,diseaseName,disease_mention,DISEASE_ENTITY_OFFSET,raw_sentence,diseaseId

+----+----------+-------------------+--------------+---------------------------------+----------+----------------+----------------------+---------------------------------------------+-------------------+-------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------+
|    |   GAD_ID | associationType   | geneSymbol   | GAD_GENE_NAME                   |   geneId | gene_mention   | GENE_ENTITY_OFFSET   | diseaseName                                 | disease_mention   | DISEASE_ENTITY_OFFSET   | raw_sentence                                                                                                                                                        | diseaseId    |
+====+==========+

In [17]:
import pandas as pd
from tabulate import tabulate

# Path to the CSV file
ftrain = '../data/GAD_merged_samples_mesh.csv'

# Read the CSV file with the correct encoding
df = pd.read_csv(ftrain, encoding='latin1')

# Display the first 4 rows to check the structure
print("First 4 rows of the DataFrame:")
print(tabulate(df.head(4), headers='keys', tablefmt='grid'))

# Get unique labels from the 'associationType' column
labels = df['associationType'].unique()
print(f"\nUnique labels: {labels}")

# Filter and display samples for the label 'F'
target_label = 'Y'
if target_label in labels:
    filtered_df = df[df['associationType'] == target_label]
    print(f"\nSamples with label '{target_label}':")
    print(tabulate(filtered_df.head(8), headers='keys', tablefmt='grid'))
else:
    print(f"No samples found with label '{target_label}'")



First 4 rows of the DataFrame:
+----+----------+-------------------+--------------+---------------------------------+----------+----------------+----------------------+---------------------------------------------+-------------------+-------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------+
|    |   GAD_ID | associationType   | geneSymbol   | GAD_GENE_NAME                   |   geneId | gene_mention   | GENE_ENTITY_OFFSET   | diseaseName                                 | disease_mention   | DISEASE_ENTITY_OFFSET   | raw_sentence                                                                                                                                                        | diseaseId    |
+====+==========+===================+==============+=================================+==========+================+======================+==========

# Read Data

In [30]:
sent_contents, entity1_list, entity2_list, sent_lables,gene_id_list,disease_id_list,gene_symbol_list = dataRead_befree(trainFile)

Input File Reading


In [31]:
word_list, distance1_list, distance2_list  = get_wordList_and_distances_befree(sent_contents)

In [32]:
print ("train_size", len(word_list))


train_size 5330


In [33]:
sent_lengths= findSentLengths([word_list])

In [34]:
sentMax =max(max(sent_lengths[:]))


## Prepare Lable Matrix

In [35]:
# Three Class

# Y : positive association 
# N:  negative association
# F:  No Smmantic Association


label_dict = {'F':0, 'Y': 1, 'N': 2}


Y = mapLabelToId_befree(sent_lables, label_dict)
Y_train = np.zeros((len(Y), len(label_dict)))
for i in range(len(Y)):
    Y_train[i][Y[i]] = 1.0

# Generate Word and Position Embedding Vectors

### Word Embedding

In [36]:
word_dict, word_to_id, id_to_word = word_mapping(word_list)
print( "word dictonary length", len(word_dict))
word_vectors = readWordEmb_fastText(word_dict,id_to_word,word_to_id, wefile, embSize)
X_train =   mapWordToId(word_list, word_to_id)

Found 6766 unique words (139634 in total)
word dictonary length 6766
Reading word vectors
number of unknown word in word embedding 1525
number of known word in word embedding 5241


### Position Embedding

In [37]:
distance1_dict = makeDistanceList([distance1_list])
distance2_dict = makeDistanceList([distance2_list])
distance1_vectors = mapWordToId_list(distance1_list, distance1_dict)
distance2_vectors = mapWordToId_list(distance2_list, distance2_dict)

### Pad Embdding Vectors

In [38]:
X_train, distance1_vectors, distance2_vectors = paddData([X_train, distance1_vectors, distance2_vectors ], sentMax,padd_num= 6765) 


# Save Prepared Data as Pickle File

In [39]:
with open('../data/pickles/befree_3class_crawl-300d-2M.pickle', 'wb') as handle:
    pickle.dump(gene_id_list, handle)
    pickle.dump(gene_symbol_list, handle)
    pickle.dump(disease_id_list, handle)
    pickle.dump(X_train, handle)
    pickle.dump(distance1_vectors, handle)
    pickle.dump(distance2_vectors, handle)    
    pickle.dump(Y_train, handle)
    pickle.dump(word_list, handle)
    pickle.dump(word_vectors, handle)
    pickle.dump(word_dict, handle)
    pickle.dump(distance1_dict, handle)
    pickle.dump(distance2_dict, handle)
    pickle.dump(label_dict, handle) 
    pickle.dump(sentMax, handle)
    
    
    
    